# Import libraries

In [1]:
import numpy as np
import matplotlib as plt
import operator
from astropy.io import fits
from astropy.table import Table, Column 
import sys
import os 
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr
import ipywidgets as widgets

# Load in 2020 fitspec files and create df

In [ ]:
rootdir20 = '/Volumes/Extreme SSD/2020fitspec/'
fitspec_files20 = [os.path.join(rootdir20,f) for f in os.listdir(rootdir20) if 
                 (f.endswith('.fits'))]

moondistance20 = []
target20 = []
exp_thres20 = []
exp_count20 = []
date20 = []
filename_df20 = []


#for subdir, dirs, files in os.walk(rootdir): 
for item in fitspec_files20:

    # get filename by itself, remove path from filename
    fileName = os.path.basename(item)

    hdus = fits.open(item)

    head = hdus[0].header  

    data = hdus[1].data

    #hdr.items

    data = hdus[1].data

    if float(head['EXPCOUNT']) >= float(head['EXPTHRES']) :

        moondistance20.append(float(head['MOONDIST']))
        target20.append(head['OBJECT'])
        exp_thres20.append(float(head['EXPTHRES']))
        exp_count20.append(float(head['EXPCOUNT']))
        date20.append(float(head['TELMJD']))
        filename_df20.append(fileName)
    
    #print(head)

df_20 = pd.DataFrame({'OBJECT': target20, 'TEL MJD': date20, 'Moon Distance': moondistance20, 'EXP COUNT': 
                    exp_count20, 'EXP THRESH': exp_thres20, 'FILE NAME': filename_df20})

#print(df_20)
df_20.reset_index(drop=True).style

# Load in 2021 fitspec files

In [ ]:
rootdir21 = '/Volumes/Extreme SSD/EXPRES-OBS/2021/fitspec/'

fitspec_files21 = [os.path.join(rootdir21,f) for f in os.listdir(rootdir21) if 
                 (f.endswith('.fits'))]

moondistance21 = []
target21 = []
exp_thres21 = []
exp_count21 = []
date21 = []
filename_df21 = []


#for subdir, dirs, files in os.walk(rootdir): 
for items in fitspec_files21:

    # get filename by itself, remove path from filename
    fileNames = os.path.basename(item)

    hdu = fits.open(items)

    heads = hdu[0].header  

    dat = hdu[1].data

    #hdr.items

    dat = hdu[1].data

    if float(heads['EXPCOUNT']) >= float(heads['EXPTHRES']) :

        moondistance21.append(float(heads['MOONDIST']))
        target21.append(heads['OBJECT'])
        exp_thres21.append(float(heads['EXPTHRES']))
        exp_count21.append(float(heads['EXPCOUNT']))
        date21.append(float(heads['TELMJD']))
        filename_df21.append(fileNames)
    

df_21 = pd.DataFrame({'OBJECT': target21, 'TEL MJD': date21, 'Moon Distance': moondistance21, 'EXP COUNT': 
                    exp_count21, 'EXP THRESH': exp_thres21, 'FILE NAME': filename_df21})

#print(df_21)
df_21.reset_index(drop=True).style

# Combine 2020 & 2021 data frames

In [ ]:
frames = [df_20, df_21]

result_df = pd.concat(frames)
#print(result_df)
result_df.reset_index(drop=True).style

# Sort combined 2020 + 2021 df's. Reset index. 

In [ ]:
sorted_df = result_df.sort_values(by=['TEL MJD'], ascending=True)
sorted_df = sorted_df.reset_index(drop=True)

#print(sorted_df)
sorted_df.reset_index(drop=True).style

# Load in vels files and create df_2

In [ ]:
# Load in vels/cbc_python files

directory2 = '/Volumes/Extreme SSD/EXPRES-OBS/vels/' # cbc_python
vels_files = [os.path.join(directory2,f) for f in os.listdir(directory2) if 
                 (f.endswith('.fits'))]


date = []
rv = []
rv_err = []
filename_df2 = []


for file in vels_files: 
    #print ("---", vels_files)
    
    try:
        #print(x)

        # get filename by itself, remove path from filename
        fileNames = os.path.basename(file)

        hdus = fits.open(file) # Read fits file into hdus variable 

        #print(len(hdus))

        hdr = hdus[1].header #1

        data = hdus[1].data # 1, place data from HDU 1 into data variable
        
        if hdr['RV_ERR'] >= 0.1:
            #print(hdr)
            #print(fileNames)
            date.append(hdr['MJD'])
            rv.append(hdr['RV'])
            rv_err.append(hdr['RV_ERR'])
            filename_df2.append(fileNames) 
            
    except:
        print(fileNames + " does not contain MJD")


In [ ]:
# Sort df_v by date

df_v = pd.DataFrame({'FILE NAME': filename_df2, 'MJD': date, 'RV': rv, 'RV Error': rv_err})
#sorted_df_2 = df_2.sort_values(by=['MJD'], ascending=True)
#print(df_v)

df_v.reset_index(drop=True).style

# Left merge df and df_2 based on filename. Create merged_df

In [ ]:
merged_left = pd.merge(left=sorted_df, right=df_v, how='left', left_on='FILE NAME', right_on='FILE NAME')
pd.set_option('display.expand_frame_repr', False)

merged_df = merged_left.dropna()
#merged_df = merged_left.reset_index(drop=True)

#print(merged_df)
merged_df.reset_index(drop=True).style

In [ ]:
target_list = merged_df['OBJECT'].value_counts()  # get counts of each target

#find which objects have min 20 observations
good_targets = target_list[(target_list >= 20)] 
#print(good_targets)

good_obs = np.zeros(len(merged_df), dtype=bool) # creating empty bool array of zeros of same length as data frame

for obj in good_targets.keys():      # loop through every object name of good targets 
    good_obs = good_obs | (merged_df['OBJECT']==obj)    # or matches, 1's for 

    
new_df = merged_df[good_obs]

#print(new_df)

print(np.sum(good_obs))
#new_df.reset_index(drop=True).style

In [ ]:
new_df.reset_index(drop=True).style

In [ ]:
#good_targets.describe()

np.sum(merged_df['OBJECT'] == '166620')
np.sum(good_targets)

# Print merged_df in txt file

In [ ]:
f = open("Moon Distance df output.txt", "w") # "w" - Write will create a file if the specified file does not exist

sys.stdout = f # record everyhing after this in output file

result = new_df.to_string(index=False, header=True)

print(result)


sys.stdout.close()